In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Sat Dec  5 02:51:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [3]:
!git clone https://github.com/adapter-hub/adapter-transformers.git
%cd adapter-transformers
!pip install .
!pip install sklearn

Cloning into 'adapter-transformers'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 42535 (delta 15), reused 11 (delta 3), pack-reused 42490
Receiving objects: 100% (42535/42535), 23.90 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (30838/30838), done.
/content/adapter-transformers
Processing /content/adapter-transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
     |████████████████████████████████| 2.9MB 54.7MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-1.1.0-cp36-none-any.whl size=1326572 sha256=27ca7f8c3b1839a277c243d3a80e8a09dafac985b40d3190e4ee6fb111f03ea0
  Stored in directory: /root/.cache/pip/wheels/69/f2/bd/f0720d145cbd02c42f9d833e06bb319e9ee518b2

In [4]:
%cd ../../..

/


In [5]:
%cd content

/content


In [6]:
!git clone https://github.com/nsusanj/temp.git

Cloning into 'temp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [7]:
!ls

adapter-transformers  gdrive  sample_data  temp


In [8]:
%cd temp/adapticons
%mkdir datasets
%cd datasets/
%mkdir citation
%cd citation
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/test.jsonl
import json
import pandas as pd
data = []
with open("train.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("train.txt", header=False, index=False)
data = []
with open("dev.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("dev.txt", header=False, index=False)
%cd ../../modeling

/content/temp/adapticons
/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/citation
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  454k  100  454k    0     0   334k      0  0:00:01  0:00:01 --:--:--  334k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30138  100 30138    0     0  35331      0 --:--:-- --:--:-- --:--:-- 35331
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38637  100 38637    0     0  45455      0 --:--:-- --:--:-- --:--:-- 45401
/content/temp/adapticons/modeling


In [9]:
!python sample_pct_txt_file.py --sample_pct=0.25  --file_name=../datasets/citation/train.txt
!python sample_pct_txt_file.py --sample_pct=0.50  --file_name=../datasets/citation/train.txt
!python sample_pct_txt_file.py --sample_pct=0.75  --file_name=../datasets/citation/train.txt

Namespace(file_name='../datasets/citation/train.txt', sample_pct='0.25')
1688it [00:00, 550585.98it/s]
Namespace(file_name='../datasets/citation/train.txt', sample_pct='0.50')
1688it [00:00, 622854.33it/s]
Namespace(file_name='../datasets/citation/train.txt', sample_pct='0.75')
1688it [00:00, 647875.65it/s]


In [10]:
!ls

_25_pct.txt	  load_fusion_model.py
_50_pct.txt	  new_train.py
_75_pct.txt	  run_language_modeling_with_adapter_fusion.py
download_glue.py  run_language_modeling_with_adapters.py
glue_training.py  sample_pct_txt_file.py
jsonl_to_txt.py


In [11]:
!ls ../datasets/citation

dev.jsonl  dev.txt  test.jsonl	train.jsonl  train.txt


In [13]:
!ls ../../../gdrive/MyDrive/downsample_sciee/

_25_pct  _50_pct  _75_pct


In [14]:
!python -m run_language_modeling_with_adapters \
--train_data_file _25_pct.txt \
--line_by_line \
--eval_data_file ../datasets/citation/dev.txt \
--output_dir ../../../gdrive/MyDrive/downsample_sciee/_25_pct_adapter \
--model_type roberta-base --tokenizer_name roberta-base --mlm --per_gpu_train_batch_size 8 --gradient_accumulation_steps 32 \
--model_name_or_path roberta-base \
--do_eval \
--evaluate_during_training  \
--do_train \
--num_train_epochs 100  \
--learning_rate 0.0001 \
--logging_steps 500 \
--overwrite_output_dir \
--adapter_name citation_25_pct_adapter \

2020-12-05 02:57:23.460380: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/05/2020 02:57:26 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 02:57:26 - INFO - filelock -   Lock 140484968535488 acquired on /root/.cache/torch/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
Downloading: 100% 481/481 [00:00<00:00, 377kB/s]
12/05/2020 02:57:26 - INFO - filelock -   Lock 140484968535488 released on /root/.cache/torch/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
12/05/2020 02:57:27 - INFO - filelock -   Lock 140484968512536 acquired on /root/.cache/torch/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367

In [15]:
!python -m run_language_modeling_with_adapters \
--train_data_file _50_pct.txt \
--line_by_line \
--eval_data_file ../datasets/citation/dev.txt \
--output_dir ../../../gdrive/MyDrive/downsample_sciee/_50_pct_adapter \
--model_type roberta-base --tokenizer_name roberta-base --mlm --per_gpu_train_batch_size 8 --gradient_accumulation_steps 32 \
--model_name_or_path roberta-base \
--do_eval \
--evaluate_during_training  \
--do_train \
--num_train_epochs 100  \
--learning_rate 0.0001 \
--logging_steps 500 \
--overwrite_output_dir \
--adapter_name citation_50_pct_adapter \

Streaming output truncated to the last 5000 lines.
Iteration:  62% 68/109 [00:04<00:02, 15.18it/s]
Iteration:  64% 70/109 [00:04<00:02, 15.25it/s]
Iteration:  66% 72/109 [00:04<00:02, 16.38it/s]
Iteration:  68% 74/109 [00:04<00:02, 15.80it/s]
Iteration:  70% 76/109 [00:04<00:01, 16.57it/s]
Iteration:  72% 78/109 [00:04<00:01, 17.14it/s]
Iteration:  73% 80/109 [00:05<00:01, 16.12it/s]
Iteration:  75% 82/109 [00:05<00:01, 16.14it/s]
Iteration:  77% 84/109 [00:05<00:01, 16.05it/s]
Iteration:  79% 86/109 [00:05<00:01, 16.32it/s]
Iteration:  81% 88/109 [00:05<00:01, 17.01it/s]
Iteration:  83% 90/109 [00:05<00:01, 17.62it/s]
Iteration:  84% 92/109 [00:05<00:01, 16.98it/s]
Iteration:  86% 94/109 [00:05<00:00, 16.56it/s]
Iteration:  88% 96/109 [00:06<00:00, 15.63it/s]
Iteration:  90% 98/109 [00:06<00:00, 15.69it/s]
Iteration:  92% 100/109 [00:06<00:00, 16.47it/s]
Iteration:  94% 102/109 [00:06<00:00, 16.31it/s]
Iteration:  95% 104/109 [00:06<00:00, 15.74it/s]
Iteration:  97% 106/109 [00:06<00:

In [16]:
!python -m run_language_modeling_with_adapters \
--train_data_file _75_pct.txt \
--line_by_line \
--eval_data_file ../datasets/citation/dev.txt \
--output_dir ../../../gdrive/MyDrive/downsample_sciee/_75_pct_adapter \
--model_type roberta-base --tokenizer_name roberta-base --mlm --per_gpu_train_batch_size 8 --gradient_accumulation_steps 32 \
--model_name_or_path roberta-base \
--do_eval \
--evaluate_during_training  \
--do_train \
--num_train_epochs 100  \
--learning_rate 0.0001 \
--logging_steps 500 \
--overwrite_output_dir \
--adapter_name citation_75_pct_adapter \

Streaming output truncated to the last 5000 lines.
Epoch:  38% 38/100 [06:14<10:12,  9.87s/it]
Iteration:   0% 0/161 [00:00<?, ?it/s]
Iteration:   1% 2/161 [00:00<00:08, 18.41it/s]
Iteration:   2% 4/161 [00:00<00:08, 18.13it/s]
Iteration:   4% 7/161 [00:00<00:08, 18.45it/s]
Iteration:   6% 9/161 [00:00<00:08, 17.29it/s]
Iteration:   7% 11/161 [00:00<00:08, 17.23it/s]
Iteration:   8% 13/161 [00:00<00:08, 17.87it/s]
Iteration:   9% 15/161 [00:00<00:07, 18.30it/s]
Iteration:  11% 17/161 [00:00<00:07, 18.25it/s]
Iteration:  12% 19/161 [00:01<00:07, 18.49it/s]
Iteration:  13% 21/161 [00:01<00:08, 16.75it/s]
Iteration:  14% 23/161 [00:01<00:07, 17.52it/s]
Iteration:  16% 25/161 [00:01<00:08, 16.10it/s]
Iteration:  17% 27/161 [00:01<00:08, 16.55it/s]
Iteration:  18% 29/161 [00:01<00:08, 16.12it/s]
Iteration:  19% 31/161 [00:01<00:07, 16.49it/s]
Iteration:  20% 33/161 [00:01<00:07, 16.38it/s]
Iteration:  22% 35/161 [00:02<00:07, 16.90it/s]
Iteration:  23% 37/161 [00:02<00:07, 17.04it/s]
Iterat